Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [11]:
batch_size = 128
lr_reg = 0.01

graph_lr = tf.Graph()
with graph_lr.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
      tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + lr_reg*tf.nn.l2_loss(weights)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
num_steps = 3001

with tf.Session(graph=graph_lr) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
batch_size = 128
n_hidden = 1024
nn_reg = 0.01
graph_nn = tf.Graph()
with graph_nn.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables (output layer)
    weights_o = tf.Variable(
                            tf.truncated_normal([n_hidden, num_labels]))
    biases_o = tf.Variable(tf.zeros([num_labels]))
  
    # Variables (hidden layer)
    weights_h = tf.Variable(
                            tf.truncated_normal([image_size * image_size, n_hidden]))
    biases_h = tf.Variable(tf.zeros([n_hidden]))
  
    # Training computation.
    hidden = tf.matmul(tf_train_dataset, weights_h) + biases_h
    hidden = tf.nn.relu(hidden)
    logits = tf.matmul(hidden, weights_o) + biases_o
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+ nn_reg*(tf.nn.l2_loss(weights_o)+tf.nn.l2_loss(weights_h))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights_h)+biases_h), weights_o) + biases_o)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_h)+biases_h), weights_o) + biases_o)    
    
    

with tf.Session(graph=graph_nn) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))    

Initialized
Minibatch loss at step 0: 48.001392
Minibatch accuracy: 4.7%
Validation accuracy: 11.7%
Minibatch loss at step 500: 1.076731
Minibatch accuracy: 78.9%
Validation accuracy: 81.5%
Minibatch loss at step 1000: 0.776055
Minibatch accuracy: 82.8%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 0.778135
Minibatch accuracy: 83.6%
Validation accuracy: 82.0%
Minibatch loss at step 2000: 0.624960
Minibatch accuracy: 87.5%
Validation accuracy: 81.5%
Minibatch loss at step 2500: 0.610967
Minibatch accuracy: 88.3%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 0.621112
Minibatch accuracy: 87.5%
Validation accuracy: 81.8%
Test accuracy: 88.5%
Initialized
Minibatch loss at step 0: 3523.439209
Minibatch accuracy: 7.0%
Validation accuracy: 27.5%
Minibatch loss at step 500: 21.322632
Minibatch accuracy: 79.7%
Validation accuracy: 84.9%
Minibatch loss at step 1000: 0.832946
Minibatch accuracy: 82.0%
Validation accuracy: 83.7%
Minibatch loss at step 1500: 0.717184
Minibatch ac

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

Real overfit problem if no regularisation used

In [13]:
batch_size = 128
lr_reg = 0.01*0

graph_lr = tf.Graph()
with graph_lr.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
      tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + lr_reg*tf.nn.l2_loss(weights)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
num_steps = 3001

with tf.Session(graph=graph_lr) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = ((step * batch_size) % (train_labels.shape[0] - batch_size)) % 10
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss at step 0: 14.176755
Minibatch accuracy: 19.5%
Validation accuracy: 13.5%
Minibatch loss at step 500: 0.004899
Minibatch accuracy: 100.0%
Validation accuracy: 52.8%
Minibatch loss at step 1000: 0.002178
Minibatch accuracy: 100.0%
Validation accuracy: 54.0%
Minibatch loss at step 1500: 0.001428
Minibatch accuracy: 100.0%
Validation accuracy: 54.5%
Minibatch loss at step 2000: 0.001009
Minibatch accuracy: 100.0%
Validation accuracy: 55.0%
Minibatch loss at step 2500: 0.000810
Minibatch accuracy: 100.0%
Validation accuracy: 55.3%
Minibatch loss at step 3000: 0.000678
Minibatch accuracy: 100.0%
Validation accuracy: 55.5%
Test accuracy: 61.3%


Regularisation saves us some pain

In [14]:
batch_size = 128
lr_reg = 0.01

graph_lr = tf.Graph()
with graph_lr.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
      tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + lr_reg*tf.nn.l2_loss(weights)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
num_steps = 3001

with tf.Session(graph=graph_lr) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss at step 0: 43.674793
Minibatch accuracy: 10.9%
Validation accuracy: 15.9%
Minibatch loss at step 500: 1.122305
Minibatch accuracy: 78.9%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 0.776789
Minibatch accuracy: 82.8%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 0.778041
Minibatch accuracy: 83.6%
Validation accuracy: 82.0%
Minibatch loss at step 2000: 0.624961
Minibatch accuracy: 87.5%
Validation accuracy: 81.5%
Minibatch loss at step 2500: 0.610965
Minibatch accuracy: 88.3%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 0.621114
Minibatch accuracy: 87.5%
Validation accuracy: 81.8%
Test accuracy: 88.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [16]:
batch_size = 128
lr_reg = 0.01*0
dropout = 0.5

graph_lr = tf.Graph()
with graph_lr.as_default():
    
    keep_prob = tf.placeholder(tf.float32)
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
      tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf.nn.dropout(tf_train_dataset,keep_prob), weights) + biases
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + lr_reg*tf.nn.l2_loss(weights)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
num_steps = 3001

with tf.Session(graph=graph_lr) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = ((step * batch_size) % (train_labels.shape[0] - batch_size)) % 10
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob:dropout}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss at step 0: 21.198105
Minibatch accuracy: 14.1%
Validation accuracy: 18.5%
Minibatch loss at step 500: 0.151179
Minibatch accuracy: 95.3%
Validation accuracy: 72.7%
Minibatch loss at step 1000: 0.218221
Minibatch accuracy: 96.9%
Validation accuracy: 72.5%
Minibatch loss at step 1500: 0.004678
Minibatch accuracy: 100.0%
Validation accuracy: 72.4%
Minibatch loss at step 2000: 0.000020
Minibatch accuracy: 100.0%
Validation accuracy: 72.5%
Minibatch loss at step 2500: 0.152725
Minibatch accuracy: 99.2%
Validation accuracy: 72.7%
Minibatch loss at step 3000: 0.001380
Minibatch accuracy: 100.0%
Validation accuracy: 72.4%
Test accuracy: 79.4%


Dropout greatly helps with the overfitting, to the same order of benefit as regularisation. Unsure which has more benefit - could be a question of hyperparameter tuning

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [41]:
batch_size = 128
n_hidden1 = 1024
n_hidden2 = 50
nn_reg = 0.1
test_dataset = test_dataset.astype('float32')
train_dataset = train_dataset.astype('float32')
valid_dataset= valid_dataset.astype('float32')


graph_nn = tf.Graph()
with graph_nn.as_default():
    

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables (output layer)
    weights_o = tf.Variable(
                            tf.truncated_normal([n_hidden2, num_labels]))
    biases_o = tf.Variable(tf.zeros([num_labels]))
  
    # Variables (hidden layer1)
    weights_h1 = tf.Variable(
                            tf.truncated_normal([image_size * image_size, n_hidden1]))
    biases_h1 = tf.Variable(tf.zeros([n_hidden1]))

    # Variables (hidden layer2)
    weights_h2 = tf.Variable(
                            tf.truncated_normal([n_hidden1, n_hidden2]))
    biases_h2 = tf.Variable(tf.zeros([n_hidden2]))


    
    # Training computation.
    hidden1 = tf.matmul(tf_train_dataset, weights_h1) + biases_h1
    hidden1 = tf.nn.relu(hidden1)
    hidden2 = tf.matmul(hidden1, weights_h2) + biases_h2
    hidden2 = tf.nn.relu(hidden2)
    logits = tf.matmul(hidden2, weights_o) + biases_o
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+ nn_reg*(tf.nn.l2_loss(weights_o)+
                                                                               tf.nn.l2_loss(weights_h1)+
                                                                              tf.nn.l2_loss(weights_h2)
                                                                              )
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    #global_step = tf.Variable(0)  # count the number of steps taken.
    #learning_rate = tf.train.exponential_decay(0.1, global_step,100,0.9)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
                        tf.matmul(
                        tf.nn.relu(
                        tf.matmul(
                        tf.nn.relu(
                        tf.matmul(
                                tf_valid_dataset,weights_h1)+biases_h1),
                        weights_h2)+
                        biases_h2),
                        weights_o) +
                        biases_o)
    test_prediction = tf.nn.softmax(
                        tf.matmul(
                        tf.nn.relu(
                        tf.matmul(
                        tf.nn.relu(
                        tf.matmul(
                                tf_test_dataset,weights_h1)+biases_h1),
                        weights_h2)+biases_h2),
                        weights_o) + biases_o)
    
    

with tf.Session(graph=graph_nn) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 250 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))    

Initialized
Minibatch loss at step 0: 34653.257812
Minibatch accuracy: 6.2%
Validation accuracy: 19.5%
Minibatch loss at step 100: 12158.335938
Minibatch accuracy: 19.5%
Validation accuracy: 15.7%
Minibatch loss at step 200: 4462.689453
Minibatch accuracy: 28.1%
Validation accuracy: 25.0%
Minibatch loss at step 300: 1638.559570
Minibatch accuracy: 38.3%
Validation accuracy: 41.2%
Minibatch loss at step 400: 602.074890
Minibatch accuracy: 58.6%
Validation accuracy: 55.0%
Minibatch loss at step 500: 221.891785
Minibatch accuracy: 64.8%
Validation accuracy: 69.6%
Minibatch loss at step 600: 82.285843
Minibatch accuracy: 77.3%
Validation accuracy: 76.0%
Minibatch loss at step 700: 31.336655
Minibatch accuracy: 74.2%
Validation accuracy: 75.8%
Minibatch loss at step 800: 12.614679
Minibatch accuracy: 75.8%
Validation accuracy: 77.1%
Minibatch loss at step 900: 5.559041
Minibatch accuracy: 85.2%
Validation accuracy: 77.0%
Minibatch loss at step 1000: 3.223544
Minibatch accuracy: 77.3%
Valida

Learning rate needs to be tuned to avoid nan issues during training...